In [1]:
from tqdm import tqdm
from hitbtc import Hitbtc
from urllib.request import urlopen
from bittrex.bittrex import Bittrex
from bitfinex import Bitfinex
import krakenex
from binance.client import Client

exchanges = {}
btc = 0.01

## Read from HitBTC

In [2]:
hitbtcClient = Hitbtc(None, None)

tickers = hitbtcClient.get_tickers()
currencies = hitbtcClient.get_currencies()
symbols = hitbtcClient.get_symbol('')

hitbtc = {}

pbar = tqdm(range(len(tickers)))
pbar.set_description('Processing')

for j in pbar:
    tick = tickers[j]
    
    for d in currencies:
        if d['id'] == tick['symbol'][0:-3]:
            currency = d
            
    for d in symbols:
        if d['id'] == tick['symbol']:
            symbol = d
    
    try:
        ask = float(tick['ask'])
    except:
        ask = 0.0
    try:
        bid = float(tick['bid'])
    except:
        bid = 0.0
        
    hitbtc[tick['symbol']]={'ask':ask,'bid':bid, 'base':tick['symbol'][-3:],
                            'payinEnabled':currency['payinEnabled'],
                            'payoutEnabled':currency['payoutEnabled'],
                            'takerFee':symbol['takeLiquidityRate'],
                           'withdrawalFee':None}
    # TODO - withdraw fee not yet implemented on API
        
exchanges['hitbtc'] = hitbtc

Processing: 100%|███████████████████████████████████████████████████████████████████████████████| 428/428 [00:00<00:00, 5074.20it/s]


## Read from Bittrex

In [3]:
bittrexClient = Bittrex(None, None)  # or defaulting to v1.1 as Bittrex(None, None)
symbols = bittrexClient.get_market_summaries()['result']
currencies = bittrexClient.get_currencies()['result']

bittrex = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]
    
    names = symbol['MarketName'].split('-')
    
    # Get withdrawal fee
    for d in currencies:
        if d['Currency'] == names[1]:
            currency = d

    try:
        ask = float(symbol['Ask'])
    except:
        ask = 0.0
    try:
        bid = float(symbol['Bid'])
    except:
        bid = 0.0
    
    # Casos a parte
    if names[1] == 'BCC':
        names[1] = 'BCH'
        
    bittrex[names[1]+names[0]] = {'ask':ask,'bid':bid, 'base':names[0],
                                  'payinEnabled':True,
                                  'payoutEnabled':True,
                                 'takerFee':'0.0025',
                                 'withdrawalFee': currency['TxFee']}
    
exchanges['bittrex'] = bittrex

Processing: 100%|██████████████████████████████████████████████████████████████████████████████| 271/271 [00:00<00:00, 11496.24it/s]


## Read from Binance

In [4]:
binanceClient = Client("", "")

symbols = binanceClient.get_orderbook_tickers()

products = binanceClient.get_products()['data']

binance = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]
    
    # TODO - Get withdrawal fee - Not working
    for d in products:
        if d['symbol'] == symbol['symbol']:
            currency = d
    
    try:
        ask = float(symbol['askPrice'])
    except:
        ask = 0.0
    try:
        bid = float(symbol['bidPrice'])
    except:
        bid = 0.0
        
    if 'BCC' in symbol['symbol']:
            symbol['symbol'] = symbol['symbol'].replace('BCC', 'BCH')
        
    binance[symbol['symbol']] = {'ask':ask,'bid':bid, 'base':symbol['symbol'][-3:],
                                 'payinEnabled':True,
                                 'payoutEnabled':True,
                                 'takerFee':'0.001',
                                 'withdrawalFee': currency['withdrawFee']}
        
exchanges['binance'] = binance

Processing: 100%|██████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 10159.17it/s]


## All possible trades

In [5]:
trades = list()

for exchangeVenda in exchanges:
    for coin in exchanges[exchangeVenda]:
        for exchangeCompra in exchanges:
            if exchangeVenda != exchangeCompra:
                try:                   
                    if exchanges[exchangeCompra][coin]['base'] == exchanges[exchangeVenda][coin]['base']:                      
                        base = exchanges[exchangeVenda][coin]['base']
                        
                        priceSell = exchanges[exchangeVenda][coin]['ask'] # Price exchange is selling
                        priceBuy = exchanges[exchangeCompra][coin]['bid'] # Price exchange is buying
                        sellFee = float(exchanges[exchangeVenda][coin]['takerFee']) # Fee for the selling exchange
                        buyFee = float(exchanges[exchangeCompra][coin]['takerFee']) # Fee for the buying exchange
                        
                        buy = btc/priceSell * (1-sellFee)       # Quantity of coin bought
                        transfer = buy*(0.99)                   # Quantity of coin transfered after withdraw fee
                        sell = transfer * priceBuy * (1-buyFee) # Quantity of coin sold
                        transferBTC = sell*(0.99)               # Quantity of coin transfered back after withdraw fee

                        gain = (transferBTC-btc)/btc            # Final gain
                        
                        if gain > 0.05:
                            considerWithdraw = True            # False to show the coins with wallets in maintenance
                            
                            if considerWithdraw == True and \
                                exchanges[exchangeVenda][coin]['payoutEnabled'] == True and \
                                exchanges[exchangeCompra][coin]['payinEnabled'] == True:
                                trades.append({'coin':coin.replace(base, ''),
                                           'sell':priceSell,
                                           'buy':priceBuy,
                                           'gain':int(gain*10000)/100,
                                           'from':exchangeVenda,
                                           'to':exchangeCompra,
                                           'base':base})
                            elif considerWithdraw == False:
                                trades.append({'coin':coin.replace(base, ''),
                                           'sell':priceSell,
                                           'buy':priceBuy,
                                           'gain':int(gain*10000)/100,
                                           'from':exchangeVenda,
                                           'to':exchangeCompra,
                                           'base':base})
                except:
                    ibis = 1
    
trades = sorted(trades, key=lambda k: k['gain'],reverse=True) 

for i in trades:
    print(i)

{'coin': 'SC', 'sell': 2.05e-06, 'buy': 3.511e-06, 'gain': 67.27, 'from': 'bittrex', 'to': 'hitbtc', 'base': 'BTC'}
{'coin': 'XDN', 'sell': 1.3599e-06, 'buy': 1.63e-06, 'gain': 17.06, 'from': 'hitbtc', 'to': 'bittrex', 'base': 'BTC'}
{'coin': 'EMC', 'sell': 0.00018726, 'buy': 0.0002104, 'gain': 9.73, 'from': 'bittrex', 'to': 'hitbtc', 'base': 'BTC'}


In [11]:
# HitBTC
orderbook = hitbtcClient.get_ticker('XDNBTC')
venda = float(orderbook['ask'])
print('Preço de venda:', venda)

# Bittrex
orderbook = bittrexClient.get_ticker('BTC-XDN')['result']
compra = float(orderbook['Bid'])
print('Preço de compra:', compra)

# Quantidade Inicial
btc = 0.01
buy = btc/venda * (1-0.25/100)
print(buy)
transfer = buy - 0.01
sell = transfer * compra * (1-0.25/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

Preço de venda: 1.3872e-06
Preço de compra: 1.62e-06
7190.743944636678
Ganho: 7.19


In [8]:
# HitBTC
orderbook = hitbtcClient.get_ticker('SCBTC')
compra = float(orderbook['bid'])
print('Preço de compra:', compra)

# Bittrex
orderbook = bittrexClient.get_ticker('BTC-SC')['result']
venda = float(orderbook['Ask'])
print('Preço de venda:', venda)

# Quantidade Inicial
btc = 0.1
buy = btc/venda * (1-0.25/100)
print(buy)
transfer = buy - 0.01
sell = transfer * compra * (1-0.1/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

Preço de compra: 3.511e-06
Preço de venda: 2.05e-06
48658.53658536586
Ganho: 69.76


In [6]:
# Bittrex
orderbook = bittrexClient.get_orderbook('BTC-DGB')['result']
venda = float(orderbook['sell'][0]['Rate'])
quantidadeVenda = float(orderbook['sell'][0]['Quantity'])
print('Preço de venda:', venda)
print('Quantidade venda:', quantidadeVenda)

# HitBTC
orderbook = hitbtcClient.get_orderbook('DGBBTC')
compra = float(orderbook['bid'][0]['price'])
quantidadeCompra = float(orderbook['bid'][0]['size'])
print('Preço de compra:', compra)
print('Quantidade compra:', quantidadeCompra)

minVC = min(quantidadeCompra, quantidadeVenda)
print('Minimo de transação: ', minVC)

# Quantidade Inicial
btc = 0.09

buy = btc/venda * (1-0.25/100)
print(buy)
transfer = buy - 0.2
sell = transfer * compra * (1-0.25/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print((transferBTC-btc)*13000)

print('Ganho:', int(gain * 10000)/100)

Preço de venda: 4.41e-06
Quantidade venda: 47664.63068182
Preço de compra: 4.4e-06
Quantidade compra: 476100.0
Minimo de transação:  47664.63068182
20357.142857142855
-20.19391140000007
Ganho: -1.72


In [7]:
# Bittrex
orderbook = bittrexClient.get_ticker('BTC-DGB')['result']
venda = float(orderbook['Ask'])
print('Preço de venda:', venda)

# HitBTC
orderbook = hitbtcClient.get_ticker('DGBBTC')
compra = float(orderbook['bid'])
print('Preço de compra:', compra)

# Quantidade Inicial
btc = 0.09
buy = btc/venda * (1-0.25/100)
print(buy)
transfer = buy - 0.2
sell = transfer * compra * (1-0.25/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

Preço de venda: 4.41e-06
Preço de compra: 4.4e-06
20357.142857142855
Ganho: -1.72
